In [1]:
from pybrain.datasets            import ClassificationDataSet
from pybrain.utilities           import percentError
from pybrain.tools.shortcuts     import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure.modules   import SoftmaxLayer
from pybrain.tools.xml.networkwriter import NetworkWriter
from pybrain.tools.xml.networkreader import NetworkReader
import csv, os
from sklearn.metrics import confusion_matrix

In [2]:
csvfile = open("ImageCommand.csv", 'rb') 
reader = csv.reader(csvfile, delimiter='#',quotechar='|', quoting=csv.QUOTE_MINIMAL)

ds = ClassificationDataSet(150*75, 1, nb_classes=4)

for row in reader:
    pixels = row[0].strip("[").strip("]").split(",")
    pixels = [value.strip() for value in pixels]
    target = row[1]
    if(target == '0' or target == '2'):
        ds.addSample(pixels, target)    

In [3]:
test_data, train_data = ds.splitWithProportion(0.25)
test_data._convertToOneOfMany()
train_data._convertToOneOfMany()

In [14]:
test_data['target']

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0,

In [15]:
zero = 0
two = 0
others = 0 
for i in ds['target']:
    if i[0] == 0 and zero<310:
        zero+=1
    elif i[0] == 2:
        two+=1
    else:
        others+=1
print zero
print two
print others

310
343
157


In [16]:
if os.path.isfile('network.xml'): 
    fnn = NetworkReader.readFrom('network.xml') 
else:
    fnn = buildNetwork(train_data.indim, 100 , train_data.outdim, outclass=SoftmaxLayer)
trainer = BackpropTrainer(fnn, dataset=train_data, momentum=0.1, learningrate=0.01 , verbose=True, weightdecay=0.01) 

In [19]:
os.system("espeak 'Starting' ")
#trainer.trainUntilConvergence()
error=1
while(error>=0.02):
    error = trainer.train()
os.system("espeak 'Done' ")

Total error: 0.0167906233597


0

In [20]:
print  percentError( trainer.testOnClassData(), train_data['class'] )
print  percentError( trainer.testOnClassData(dataset=test_data ), test_data['class'] )

7.89473684211
11.3861386139


In [8]:
NetworkWriter.writeToFile(fnn, 'network.xml')

In [22]:
confusion_matrix(test_data['class'], trainer.testOnClassData(dataset=test_data ))

array([[103,   6],
       [ 17,  76]])